In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
btc_price_dir = 'Data/btc/'
btc_files = os.listdir(btc_price_dir)

files_list = []
for x in btc_files:
    if '1m' in x:
        files_list.append(x)
files_list.sort()

df_list = []
for x in files_list:
    file = pd.read_csv(btc_price_dir+x, header= None, index_col=False)
    df_list.append(file)

price_df = pd.concat(df_list,axis  = 0).drop(11,axis  = 1)
price_df.columns = ['date','open','high','low','close','close_time','volume','n_trades','taker_base','taker_quote','drop']
price_df['date'] = pd.to_datetime(price_df['date'], unit='ms')
price_df = price_df.drop(['close_time','drop','taker_base','taker_quote'],axis  = 1)
price_df.reset_index().drop('index', axis  = 1)


,date,open,high,low,close,volume,n_trades
0,2022-08-01 00:00:00,23296.36,23296.78,23262.27,23280.77,1659312059999,4.715691e+06
1,2022-08-01 00:01:00,23280.77,23317.68,23263.01,23313.09,1659312119999,3.580741e+06
2,2022-08-01 00:02:00,23316.78,23317.72,23271.23,23287.89,1659312179999,2.792681e+06
3,2022-08-01 00:03:00,23287.89,23291.08,23263.02,23278.59,1659312239999,2.341383e+06
4,2022-08-01 00:04:00,23275.48,23297.48,23266.27,23277.42,1659312299999,1.740367e+06
...,...,...,...,...,...,...,...
25915,2022-08-18 23:55:00,23204.63,23210.91,23192.36,23194.39,1660866959999,3.055828e+06
25916,2022-08-18 23:56:00,23196.32,23214.66,23185.69,23209.62,1660867019999,3.056339e+06
25917,2022-08-18 23:57:00,23209.62,23211.76,23196.12,23198.87,1660867079999,3.924531e+06
25918,2022-08-18 23:58:00,23197.84,23201.19,23185.65,23197.29,1660867139999,2.080056e+06


In [3]:
tweet_df = pd.read_csv('new_df_btc.csv', engine  = 'python').drop('Unnamed: 0',axis = 1)
tweet_df['date'] = pd.to_datetime(tweet_df['date'])


In [4]:
crypto_tweets_df = pd.merge_asof(
    tweet_df.sort_values(by = 'date'),
    price_df.sort_values(by = 'date'),
    left_on = 'date', 
    right_on = 'date', direction = 'nearest')


In [5]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()


In [6]:
crypto_tweets_df['compound'] = crypto_tweets_df.apply(lambda row: analyzer.polarity_scores(row.tweet)['compound'], axis = 1)


In [7]:
crypto_tweets_df['percent_change'] = (crypto_tweets_df['close'] -  crypto_tweets_df['open']) / crypto_tweets_df['open']
crypto_tweets_df

,conversation_id,date,tweet,language,username,nlikes,nreplies,nretweets,open,high,low,close,volume,n_trades,compound,percent_change
0,1.553954e+18,2022-08-01 00:00:00,Web3 : une université de Hong Kong va lancer d...,french,EZCnews,2.0,0.0,2.0,23296.36,23296.78,23262.27,23280.77,1659312059999,4.715691e+06,0.0000,-0.000669
1,1.553954e+18,2022-08-01 00:00:00,Sell Bitcoin $BTC @ 23375.62,english,BTC_AI_bot,1.0,0.0,0.0,23296.36,23296.78,23262.27,23280.77,1659312059999,4.715691e+06,0.0000,-0.000669
2,1.553954e+18,2022-08-01 00:00:00,"1 $BTC = $23,356.30 $USD 🔴 -1.62% chg/24h #Bi...",vietnamese,BTCHourlyPrice,1.0,0.0,0.0,23296.36,23296.78,23262.27,23280.77,1659312059999,4.715691e+06,0.0000,-0.000669
3,1.553954e+18,2022-08-01 00:00:00,AI trading session based on the DeepR BTC pred...,english,DeepR_BTC,0.0,0.0,0.0,23296.36,23296.78,23262.27,23280.77,1659312059999,4.715691e+06,0.4404,-0.000669
4,1.553954e+18,2022-08-01 00:00:00,You can claim #bitFlowers (PETAL) coin faucet ...,english,eFlower_power,0.0,0.0,0.0,23296.36,23296.78,23262.27,23280.77,1659312059999,4.715691e+06,-0.4574,-0.000669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941643,1.558183e+18,2022-08-18 17:49:49,@brettinthewoods Nah read it again. You post a...,english,solitarybutter,0.0,0.0,0.0,23341.93,23345.17,23328.88,23333.88,1660845059999,2.913119e+06,-0.3818,-0.000345
941644,1.560383e+18,2022-08-18 17:49:50,Worried about the state of the crypto market? ...,english,MarinaBracket11,0.0,0.0,0.0,23341.93,23345.17,23328.88,23333.88,1660845059999,2.913119e+06,0.6486,-0.000345
941645,1.560383e+18,2022-08-18 17:49:50,Bitcoin feels a little better today than it di...,english,nverduint,0.0,0.0,0.0,23341.93,23345.17,23328.88,23333.88,1660845059999,2.913119e+06,0.4402,-0.000345
941646,1.560383e+18,2022-08-18 17:49:53,@OTC_Bitcoin Ayo where the fuck the rest of th...,english,thealexgreen69,0.0,0.0,0.0,23341.93,23345.17,23328.88,23333.88,1660845059999,2.913119e+06,-0.7906,-0.000345


In [8]:
crypto_tweets_df.to_csv('btc_tweets.csv',index_label=False)

In [9]:
crypto_tweets_df.language.unique()

array(['french', 'english', 'vietnamese', 'japanese', 'welsh', 'german',
       'indonesian', 'polish', 'spanish', 'lithuanian', 'dutch',
       'turkish', 'thai', 'swedish', 'portugese', 'italian', 'chinese',
       'tagalog', 'arabic', 'persian', 'korean', 'estonian', 'hungarian',
       'russian', 'haitian creol', 'romanian', 'bulgarian', 'norwegian',
       'hindi', 'greek', 'danish', 'finnish', 'czech', 'slovenian',
       'urdu', 'sinhala', 'nepali', 'gujarati', 'latvian', 'icelandic',
       'tamil', 'marathi', 'malayalam', 'bangla', 'sindhi'], dtype=object)

In [10]:
sum_col = ['nlikes', 'nreplies', 'nretweets']
count_col = ['tweet']
mean_col = ['open', 'high', 'low', 'close', 'volume', 'n_trades', 'compound', 'percent_change']

In [12]:
freq = '1min'
count_df = crypto_tweets_df.groupby(pd.Grouper(key='date', axis=0, freq=freq)).count()[count_col]
count_df.columns = ['tweets_per_min']

sum_df = crypto_tweets_df.groupby(pd.Grouper(key='date', axis=0, freq=freq)).sum()[sum_col]
sum_df.columns = ['nlikes_per_min', 'nreplies_per_min','nretweets_per_min']

mean_df = crypto_tweets_df.groupby(pd.Grouper(key='date', axis=0, freq=freq)).mean()[mean_col]
mean_df.columns = ['open_avg', 'high_avg', 'low_avg', 'close_avg', 'volume_avg', 'n_trades_avg', 'compound_avg', 'percent_change_avg']

minute_df = pd.concat([count_df,sum_df,mean_df], axis = 1).reset_index()
minute_df['up_down'] = np.where(minute_df['percent_change_avg']>0,True,False)


In [13]:
minute_df

,date,tweets_per_min,nlikes_per_min,nreplies_per_min,nretweets_per_min,open_avg,high_avg,low_avg,close_avg,volume_avg,n_trades_avg,compound_avg,percent_change_avg,up_down
0,2022-08-01 00:00:00,93,675.0,251.0,141.0,23293.845484,23300.150968,23262.389355,23285.982903,1.659312e+12,4.532634e+06,0.090805,-0.000337,False
1,2022-08-01 00:01:00,31,402.0,40.0,24.0,23292.386129,23317.692903,23265.661613,23304.960968,1.659312e+12,3.326528e+06,0.092610,0.000541,True
2,2022-08-01 00:02:00,27,25.0,2.0,1.0,23305.010000,23306.866667,23267.885185,23284.101111,1.659312e+12,2.608819e+06,0.083689,-0.000897,False
3,2022-08-01 00:03:00,28,48.0,19.0,16.0,23282.128214,23294.051429,23264.528929,23278.046786,1.659312e+12,2.062340e+06,0.174514,-0.000175,False
4,2022-08-01 00:04:00,30,223.0,38.0,22.0,23275.090667,23287.496000,23265.038000,23272.566667,1.659312e+12,1.628960e+06,0.011877,-0.000108,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25545,2022-08-18 17:45:00,48,23.0,13.0,4.0,23368.053333,23377.515000,23358.281250,23373.707500,1.660845e+12,3.323523e+06,0.132142,0.000242,True
25546,2022-08-18 17:46:00,44,101.0,25.0,18.0,23371.029091,23373.648864,23356.347727,23364.096136,1.660845e+12,1.828199e+06,0.120232,-0.000297,False
25547,2022-08-18 17:47:00,41,167.0,42.0,34.0,23365.504390,23368.814146,23355.242195,23362.293902,1.660845e+12,1.553845e+06,0.144463,-0.000137,False
25548,2022-08-18 17:48:00,43,13.0,9.0,2.0,23363.190000,23365.108372,23349.440233,23351.734651,1.660845e+12,1.481432e+06,0.132098,-0.000490,False


In [14]:
freq = '1H'
count_df = crypto_tweets_df.groupby(pd.Grouper(key='date', axis=0, freq=freq)).count()[count_col]
count_df.columns = ['tweets_per_min']

sum_df = crypto_tweets_df.groupby(pd.Grouper(key='date', axis=0, freq=freq)).sum()[sum_col]
sum_df.columns = ['nlikes_per_min', 'nreplies_per_min','nretweets_per_min']

mean_df = crypto_tweets_df.groupby(pd.Grouper(key='date', axis=0, freq=freq)).mean()[mean_col]
mean_df.columns = ['open_avg', 'high_avg', 'low_avg', 'close_avg', 'volume_avg', 'n_trades_avg', 'compound_avg', 'percent_change_avg']

hour_df = pd.concat([count_df,sum_df,mean_df], axis = 1).reset_index()
hour_df['up_down'] = np.where(hour_df['percent_change_avg']>0,True,False)

In [15]:
minute_df.to_csv('minute_df_18Aug.csv',index_label=False)
hour_df.to_csv('hour_df_18Aug.csv',index_label=False)

In [17]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize = (30,8))

bar = sns.barplot(data  = minute_df, x = 'date', y = 'percent_change_avg', hue = 'up_down', palette = ['r','g'])
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.xticks(rotation = 30)
plt.show()